In [ ]:
import glob
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'  # 也可以使用 tensorflow
# os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,exception_verbosity=high'
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cuda,optimizer=fast_compile'
os.environ["CUDA_VISIBLE_DEVICES"] = "8"

import keras.backend as K

K.set_image_data_format('channels_last')

import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, LSTM, Concatenate, Dense, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten, UpSampling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.datasets import mnist
from urllib.request import urlretrieve
from keras.optimizers import RMSprop, SGD, Adam, sgd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.models import model_from_json

# from read_data import *
a = % pwd
print(a)

% run/root/jupyter/inspace/sang-min/myo_proejct/load_data.py import DataLoader

# %load load_data import DataLoader
# from load_data import DataLoader


'''
Model structure

Input : (100) Vector
Output : (64, 64, 1) image

Generator
100 -> 256
256 -> 16 x 16
16 x 16 -> 32 x 32
32 x 32 -> 64 x 64
64 x 64 -> 128 x 128

Discriminator
128 x 128 -> 64 x 64
64 x 64 -> 32 x 32
32 x 32 -> 16 x 16
16 x 16 -> 8 x 8
8 x 8 -> 64
64 -> 32 (16, 10)

'''

class MYO_GAN():
    def __init__(self):
        self.conv_init = RandomNormal(0, 0.02)
        self.gamma_init = RandomNormal(1., 0.02)

        self.epoch = 30000
        self.time_0 = time.time()
        self.batch_size = 64

        self.d_step = 1
        self.g_step = 2

        self.g_loss_history = []
        self.d_loss_real_history = []
        self.d_loss_fake_history = []

        self.lstm_size = (300, 16)
        self.noise_size = 100
        self.image_size = 128
        self.input_size = 100
        self.image_channel = 1
        self.learning_rate = 2e-4
        self.loader = DataLoader(data_path='./dataset_2018_05_16/', is_real_image=False)

        self.discriminator_optim = sgd(lr=0.01, momentum=0.9, nesterov=True)
        self.generator_optim = Adam(lr=1e-3)
        adam = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)  # as described in appendix A of DeepMind's AC-GAN paper

        # lstm_input = Input(shape=lstm_size)
        self.noise_input = Input(shape=(self.noise_size,))
        real_image = Input(shape=(self.image_size, self.image_size, self.image_channel))

        self.net_g = self.generative_model()
        fake_image = self.net_g(self.noise_input)
        self.net_g.summary()

        self.net_d = self.discriminative_model()
        self.net_d.summary()

        combined_output = self.net_d(fake_image)
        self.combined_model = Model(inputs=[self.noise_input], outputs=[combined_output], name='combined')
        '''
        net_g, net_d, combined_model = load_model()

        net_g.summary()
        net_d.summary()

        fake_image = net_g(self.noise_input)
        '''
        self.net_g.compile(loss='binary_crossentropy', optimizer=adam)
        self.net_d.compile(loss='binary_crossentropy', optimizer=adam)
        self.net_d.trainable = False
        self.combined_model.compile(loss='binary_crossentropy', optimizer=adam)

        self.combined_model.summary()

    def generative_model(self):
        #noise_input = Input(shape=(self.noise_size,))
        
        _ = Dense(256, input_shape=(100,), activation='relu')(self.noise_input)
        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Reshape((16, 16, 1), input_shape=(256,))(_)

        _ = Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same', input_shape=(16, 16, 1))(
            _)
        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Activation(activation='relu')(_)

        _ = UpSampling2D()(_)
        _ = Conv2D(filters=256, kernel_size=3, padding='same', input_shape=(16, 16, 128))(_)
        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Activation(activation='relu')(_)

        _ = UpSampling2D()(_)
        _ = Conv2D(filters=512, kernel_size=3, padding='same', input_shape=(32, 32, 256))(_)
        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Activation(activation='relu')(_)

        _ = UpSampling2D()(_)
        _ = Conv2D(filters=256, kernel_size=3, padding='same', input_shape=(64, 64, 512))(_)
        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Activation(activation='relu')(_)

        # _ = UpSampling2D()(_)
        _ = Conv2D(filters=1, kernel_size=3, padding='same', input_shape=(128, 128, 256))(_)
        _ = Activation(activation='tanh')(_)


        return Model(inputs=self.noise_input, outputs=_)


    def discriminative_model(self):
        _ = inputs = Input(shape=(self.image_size, self.image_size, self.image_channel))

        _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(128, 128, 1))(_)
        _ = LeakyReLU(alpha=0.2)(_)

        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Conv2D(filters=512, kernel_size=(1, 1), strides=2, padding='same', input_shape=(64, 64, 256))(_)
        _ = LeakyReLU(alpha=0.2)(_)

        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(32, 32, 512))(_)
        _ = LeakyReLU(alpha=0.2)(_)

        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(16, 16, 256))(_)
        _ = LeakyReLU(alpha=0.2)(_)

        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(8, 8, 128))(_)
        _ = LeakyReLU(alpha=0.2)(_)

        _ = BatchNormalization(axis=1)(_, training=1)
        _ = Conv2D(filters=self.image_channel, kernel_size=(2, 2), strides=1, padding='same', input_shape=(4, 4, 128))(_)
        # _ = LeakyReLU(alpha=0.2)(_)

        outputs = Flatten()(_)
        outputs = Dense(1, activation='sigmoid')(outputs)

        return Model(inputs=inputs, outputs=outputs)

    def load_model(self):
        json_file = open('./model3_output/load_model/g_model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        load_g_model = model_from_json(loaded_model_json)
        # load weights into new model
        load_g_model.load_weights("./model3_output/load_model/g_model.h5")

        json_file = open('./model3_output/load_model/d_model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        load_d_model = model_from_json(loaded_model_json)
        # load weights into new model
        load_d_model.load_weights("./model3_output/load_model/d_model.h5")

        json_file = open('./model3_output/load_model/combined_model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        load_combined_model = model_from_json(loaded_model_json)
        # load weights into new model
        load_combined_model.load_weights("./model3_output/load_model/combined_model.h5")

        print("Loaded model from disk")

        return [load_g_model, load_d_model, load_combined_model]

    def sample_generation(self):
        for _ in range(num):
            noise = np.random.normal(size=[num, self.noise_size])
            gan_image = net_g.predict(noise)
            cv2.imwrite('./model3_output/image/' + 'sample image' + str(_) + '.png', gan_image[_] * 127.5)

        print("generated image")

    def save_model(self):
        self.net_g.save_weights("./model3_output/save_model/g_model.h5")
        self.net_d.save_weights("./model3_output/save_model/d_model.h5")
        self.combined_model.save_weights("./model3_output/save_model/combined_model.h5")

        g_model_json = self.net_g.to_json()
        with open("./model3_output/save_model/g_model.json", "w") as json_file:
            json_file.write(g_model_json)

        d_model_json = self.net_d.to_json()
        with open("./model3_output/save_model/d_model.json", "w") as json_file:
            json_file.write(d_model_json)

        combined_model_json = self.combined_model.to_json()
        with open("./model3_output/save_model/combined_model.json", "w") as json_file:
            json_file.write(combined_model_json)

        print("Saved model to disk")

    def train(self):
        i = 0

        while i <= self.epoch:
            # x_train = loader.get_emg_datas(batch_size)
            images = self.loader.get_images(self.batch_size)

            for _ in range(self.d_step):
                noise = np.random.normal(size=[self.batch_size, self.noise_size])

                g_z = self.net_g.predict(noise)

                d_loss_real = self.net_d.train_on_batch(images, np.random.uniform(low=0.7, high=1.2, size=self.batch_size))
                d_loss_fake = self.net_d.train_on_batch(g_z, np.random.uniform(low=0.0, high=0.3, size=self.batch_size))

                self.d_loss_real_history.append(d_loss_real)
                self.d_loss_fake_history.append(d_loss_fake)
                self.d_loss = np.sum([d_loss_fake, d_loss_real])

            for _ in range(self.g_step):
                noise = np.random.normal(size=[self.batch_size, self.noise_size])
                combined_loss = self.combined_model.train_on_batch(noise,np.random.uniform(low=0.7, high=1.2, size=self.batch_size))

                self.g_loss_history.append(combined_loss)

            print("%d [D loss real: %f] [D loss fake: %f] [D loss: %f] [G loss: %f]" % (
                i, d_loss_real, d_loss_fake, self.d_loss, combined_loss))

            if i % 500 == 0:
                gan_image = self.net_g.predict(noise)
                print("gan imaga2 : ", gan_image[0].shape)
                cv2.imwrite('./model3_output/image/' + 'fake_image' + str(i) + '.png', gan_image[0] * 127.5)
                # cv2.imwrite('./output_image3/' + 'real_image'+ str(i) + '.png', images[0] * 127.5)

            i += 1;

        self.sample_generation(32, self.net_g)

    def show_history(self):
        plt.figure(1, figsize=(16, 8))
        plt.plot(self.d_loss_real_history)
        plt.ylabel('d_loss_real')
        plt.xlabel('epoch')
        plt.legend(['train'], loc='upper left')

        plt.savefig('./model3_output/image/d_loss_real_history.png')

        plt.figure(2, figsize=(16, 8))
        plt.plot(self.d_loss_fake_history)
        plt.ylabel('d_loss_fake')
        plt.xlabel('epoch')
        plt.legend(['train'], loc='upper left')

        plt.savefig('./model3_output/image/d_loss_fake_history.png')

        # plt.show()

        plt.figure(3, figsize=(16, 8))
        plt.plot(self.g_loss_history)
        plt.ylabel('g_loss')
        plt.xlabel('epoch')
        plt.legend(['train'], loc='upper left')

        plt.savefig('./model3_output/image/g_loss_history.png')

        # plt.show()\

        plt.show()
if __name__ == '__main__':
    myo_gan = MYO_GAN()
    myo_gan.train()

    print("Finish")

'''
net_g = generative_model(noise_size, image_channel)
fake_image = net_g(noise_input)
net_g.summary()

net_d = discriminative_model(image_size, image_channel)
net_d.summary()

combined_output = net_d(fake_image)
combined_model = Model(inputs=[noise_input], outputs=[combined_output], name='combined')
'''


'''
loader = DataLoader(data_path='./MYO_Dataset_label/')

emg = loader.load_emg_data()
image, label = loader.load_image()

print(emg.shape)
print(image.shape, label)

emg = loader.get_emg_datas(10)
images, labels = loader.get_images(10)
print(emg.shape, images.shape, labels.shape)

'''


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


/root/jupyter/inspace/sang-min/myo_proejct
30
./dataset_2018_05_16/1/
15
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       1280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       64        
_________________________________________________________________
act

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss real: 1.273300] [D loss fake: 6.803737] [D loss: 8.077037] [G loss: 0.233347]
gan imaga2 :  (128, 128, 1)
1 [D loss real: 0.429120] [D loss fake: 0.863993] [D loss: 1.293113] [G loss: 3.395236]
2 [D loss real: 0.339489] [D loss fake: 0.614672] [D loss: 0.954161] [G loss: 3.036199]
3 [D loss real: 0.218471] [D loss fake: 0.398538] [D loss: 0.617009] [G loss: 1.774432]
4 [D loss real: 0.284366] [D loss fake: 0.434164] [D loss: 0.718530] [G loss: 1.682467]
5 [D loss real: 0.267459] [D loss fake: 0.405632] [D loss: 0.673092] [G loss: 1.927379]
6 [D loss real: 0.237164] [D loss fake: 0.445963] [D loss: 0.683127] [G loss: 1.821254]
7 [D loss real: 0.078805] [D loss fake: 0.433467] [D loss: 0.512272] [G loss: 1.764116]
8 [D loss real: 0.254449] [D loss fake: 0.401633] [D loss: 0.656082] [G loss: 1.942204]
9 [D loss real: 0.164572] [D loss fake: 0.396695] [D loss: 0.561267] [G loss: 1.825456]
10 [D loss real: 0.312446] [D loss fake: 0.432673] [D loss: 0.745119] [G loss: 1.858110]
11 

91 [D loss real: 0.297169] [D loss fake: 0.451246] [D loss: 0.748415] [G loss: 1.693953]
92 [D loss real: 0.105959] [D loss fake: 0.427560] [D loss: 0.533519] [G loss: 1.730070]
93 [D loss real: 0.236599] [D loss fake: 0.393190] [D loss: 0.629789] [G loss: 1.901893]
94 [D loss real: 0.265778] [D loss fake: 0.429654] [D loss: 0.695432] [G loss: 1.841251]
95 [D loss real: 0.292171] [D loss fake: 0.423985] [D loss: 0.716156] [G loss: 1.843973]
96 [D loss real: 0.184016] [D loss fake: 0.430869] [D loss: 0.614885] [G loss: 1.633896]
97 [D loss real: 0.205275] [D loss fake: 0.444770] [D loss: 0.650046] [G loss: 1.740165]
./dataset_2018_05_16/8/
98 [D loss real: 0.328446] [D loss fake: 0.481547] [D loss: 0.809993] [G loss: 1.618150]
99 [D loss real: 0.171053] [D loss fake: 0.421493] [D loss: 0.592546] [G loss: 1.957934]
100 [D loss real: 0.193126] [D loss fake: 0.430686] [D loss: 0.623812] [G loss: 1.707080]
101 [D loss real: 0.212493] [D loss fake: 0.423752] [D loss: 0.636246] [G loss: 1.762

...

721 [D loss real: 0.186113] [D loss fake: 0.427153] [D loss: 0.613266] [G loss: 1.781986]
722 [D loss real: 0.267869] [D loss fake: 0.451808] [D loss: 0.719677] [G loss: 1.725359]
723 [D loss real: 0.134692] [D loss fake: 0.429506] [D loss: 0.564198] [G loss: 1.738758]
724 [D loss real: 0.278457] [D loss fake: 0.459967] [D loss: 0.738424] [G loss: 1.744981]
725 [D loss real: 0.185685] [D loss fake: 0.428662] [D loss: 0.614348] [G loss: 1.809497]
726 [D loss real: 0.190739] [D loss fake: 0.409518] [D loss: 0.600257] [G loss: 1.808376]
727 [D loss real: 0.167393] [D loss fake: 0.418264] [D loss: 0.585658] [G loss: 1.849611]
728 [D loss real: 0.296977] [D loss fake: 0.420111] [D loss: 0.717088] [G loss: 1.768902]
729 [D loss real: 0.256342] [D loss fake: 0.411898] [D loss: 0.668240] [G loss: 1.828978]
./dataset_2018_05_16/8/
730 [D loss real: 0.243177] [D loss fake: 0.419863] [D loss: 0.663040] [G loss: 1.812092]
731 [D loss real: 0.225684] [D loss fake: 0.453079] [D loss: 0.678762] [G lo

811 [D loss real: 0.133904] [D loss fake: 0.424552] [D loss: 0.558456] [G loss: 1.849724]
812 [D loss real: 0.232950] [D loss fake: 0.408374] [D loss: 0.641324] [G loss: 1.832223]
813 [D loss real: 0.101380] [D loss fake: 0.421013] [D loss: 0.522392] [G loss: 1.846824]
./dataset_2018_05_16/14/
814 [D loss real: 0.212705] [D loss fake: 0.409537] [D loss: 0.622242] [G loss: 1.863663]
815 [D loss real: 0.239801] [D loss fake: 0.427187] [D loss: 0.666988] [G loss: 1.845382]
816 [D loss real: 0.159123] [D loss fake: 0.371991] [D loss: 0.531114] [G loss: 1.890434]
817 [D loss real: 0.176533] [D loss fake: 0.417630] [D loss: 0.594163] [G loss: 1.947944]
818 [D loss real: 0.309687] [D loss fake: 0.445476] [D loss: 0.755162] [G loss: 1.751185]
819 [D loss real: 0.265872] [D loss fake: 0.465196] [D loss: 0.731069] [G loss: 1.702731]
820 [D loss real: 0.200356] [D loss fake: 0.419428] [D loss: 0.619784] [G loss: 1.765106]
821 [D loss real: 0.215598] [D loss fake: 0.433654] [D loss: 0.649252] [G l

901 [D loss real: 0.212310] [D loss fake: 0.401655] [D loss: 0.613966] [G loss: 1.846912]
902 [D loss real: 0.130998] [D loss fake: 0.392690] [D loss: 0.523688] [G loss: 1.828327]
903 [D loss real: 0.203739] [D loss fake: 0.421184] [D loss: 0.624923] [G loss: 1.925707]
904 [D loss real: 0.139788] [D loss fake: 0.426678] [D loss: 0.566466] [G loss: 1.800565]
905 [D loss real: 0.190672] [D loss fake: 0.447273] [D loss: 0.637945] [G loss: 1.780000]
906 [D loss real: 0.176826] [D loss fake: 0.408873] [D loss: 0.585698] [G loss: 1.780201]
907 [D loss real: 0.293293] [D loss fake: 0.416144] [D loss: 0.709438] [G loss: 1.827168]
908 [D loss real: 0.215671] [D loss fake: 0.453521] [D loss: 0.669192] [G loss: 1.729436]
909 [D loss real: 0.189250] [D loss fake: 0.390863] [D loss: 0.580113] [G loss: 1.828083]
910 [D loss real: 0.091648] [D loss fake: 0.428681] [D loss: 0.520329] [G loss: 1.887994]
911 [D loss real: 0.228080] [D loss fake: 0.443020] [D loss: 0.671101] [G loss: 1.814334]
912 [D los

991 [D loss real: 0.180550] [D loss fake: 0.404762] [D loss: 0.585312] [G loss: 1.943638]
992 [D loss real: 0.187022] [D loss fake: 0.434274] [D loss: 0.621297] [G loss: 1.845617]
993 [D loss real: 0.237145] [D loss fake: 0.447276] [D loss: 0.684421] [G loss: 1.724636]
994 [D loss real: 0.218453] [D loss fake: 0.410552] [D loss: 0.629004] [G loss: 1.792433]
995 [D loss real: 0.287362] [D loss fake: 0.378124] [D loss: 0.665486] [G loss: 1.939098]
996 [D loss real: 0.258837] [D loss fake: 0.422676] [D loss: 0.681513] [G loss: 1.916975]
./dataset_2018_05_16/12/
997 [D loss real: 0.187577] [D loss fake: 0.431247] [D loss: 0.618824] [G loss: 1.881225]
998 [D loss real: 0.207258] [D loss fake: 0.389159] [D loss: 0.596417] [G loss: 1.870266]
999 [D loss real: 0.297546] [D loss fake: 0.381021] [D loss: 0.678566] [G loss: 1.941576]
1000 [D loss real: 0.208375] [D loss fake: 0.420980] [D loss: 0.629355] [G loss: 1.891995]
gan imaga2 :  (128, 128, 1)
1001 [D loss real: 0.200199] [D loss fake: 0.4

1080 [D loss real: 0.122516] [D loss fake: 0.425833] [D loss: 0.548349] [G loss: 1.897169]
./dataset_2018_05_16/3/
1081 [D loss real: 0.276081] [D loss fake: 0.424032] [D loss: 0.700113] [G loss: 1.810246]
1082 [D loss real: 0.226327] [D loss fake: 0.387790] [D loss: 0.614116] [G loss: 1.856520]
1083 [D loss real: 0.278443] [D loss fake: 0.408030] [D loss: 0.686473] [G loss: 1.904776]
1084 [D loss real: 0.283326] [D loss fake: 0.388320] [D loss: 0.671645] [G loss: 1.843569]
1085 [D loss real: 0.189533] [D loss fake: 0.427991] [D loss: 0.617524] [G loss: 1.876135]
1086 [D loss real: 0.188869] [D loss fake: 0.425997] [D loss: 0.614867] [G loss: 1.839846]
1087 [D loss real: 0.142461] [D loss fake: 0.432812] [D loss: 0.575273] [G loss: 1.774851]
1088 [D loss real: 0.127216] [D loss fake: 0.410658] [D loss: 0.537875] [G loss: 1.804924]
1089 [D loss real: 0.116157] [D loss fake: 0.443248] [D loss: 0.559404] [G loss: 1.790253]
1090 [D loss real: 0.210840] [D loss fake: 0.421406] [D loss: 0.63